In [ ]:
import re
import requests
import urllib.request as img_request
import time
from tqdm import tqdm
import pandas as pd
import json 
import matplotlib.pyplot as plt
from PIL import Image
from os import makedirs
from os import listdir
from os import getcwd
import os
from data.classes import *
import numpy as np
from bs4 import BeautifulSoup
from Labeller import Labeller
import PIL
import torchvision
import glob
import torch
import io

from sklearn.model_selection import train_test_split

# Init

Defining important directories as variables

In [ ]:
cwd = getcwd()
data_dir = "data/"
img_dir = "E:/data/images/"

Defining regex tags

In [ ]:
if not (os.path.exists(data_dir)):
    print("Making dir" , data_dir)
    makedirs(data_dir)

if not(os.path.exists(img_dir)):
    print("Making dir" , img_dir)
    makedirs(img_dir)

if not(os.path.exists(img_dir+"/boat24")):
    print("Making dir" , img_dir+"/boat24")
    makedirs(img_dir+"/boat24")

if not(os.path.exists(img_dir+"/image_search")):
    print("Making dir" , img_dir+"/image_search")
    makedirs(img_dir+"/image_search")

# Scraping

## Sailboatdata.com

In [ ]:
n_pages_tag = re.compile(r'<a href=.*page=(\d*).*/li>') # For getting the amount of pages on the domain

name_tag = re.compile(r'<a href=\"https://sailboatdata\.com/sailboat/.*\">(.*)</a>') # For making a list of all the boats on sailboatdata

specs_tag = re.compile(r'<div class=\" col-\w\w-\d*  col-\w\w-6 sailboatdata-label \">\s*(.*):\s</div>\s<.*\s*(.*)') # For scraping data from specific boat url

image_tag = re.compile(r'(?:photo|drawing)\".*src=\"(http.*)\"/>')

photo_draw_tag = re.compile(r'sailboat/(\w*)')

name_description_tag = re.compile(r'title" content="(.+)"[^½]+?description" content="(.+)"')

In [ ]:
specs = []
categories = []
boat_data = pd.DataFrame()
counter = 0

In [ ]:
general_url = 'https://sailboatdata.com/sailboat?page={}&paginate=25'

print("Connecting to" , general_url.format(1))
init_request = requests.get(general_url.format(1))
print(init_request.status_code)
pages = re.findall(n_pages_tag , init_request.text)
n_pages = pages[-1]

boat_types = []
for i in tqdm(range (1,int(n_pages)+1)):
  # print("Connecting to" , general_url.format(i))
  r = requests.get(general_url.format(i))
  # print(r.status_code)
  boat_types += re.findall(name_tag,r.text)
  time.sleep(0.5) #To not throw too many requests at the website

print(len(boat_types))

In [ ]:
for boat in boat_types:
    # GETTNG REQUEST
    print("Scraping",boat,"...")
    r = requests.get("https://sailboatdata.com/sailboat/{}?units=metric".format(boat.replace(" ","-").replace("(","").replace(")","").replace(".","")))
    raw_text = r.text

    # SCRAPING IMAGES
    images = re.findall(image_tag,raw_text)
    print("         -------scraping")
    for image in images:
        img_path = str(counter).zfill(5)+".jpg"
        if not os.path.isfile(img_path):
            counter += 1
            print(image)
            try:
                img_request.urlretrieve(image,os.path.join(img_dir,'sailboatdata')+img_path)
            except:
                print("Strange url" , image.replace(" ","%20"))
                try:
                    img_request.urlretrieve(image,img_dir+img_path)
                except:
                    has_image = False

            # SCRAPING SPECS
            categories_specs = re.findall(specs_tag,raw_text)
            specs = [el[1] for el in categories_specs]
            categories = [el[0] for el in categories_specs]
            new_row = {categories[i]: specs[i] for i in range(len(categories))}
            new_row['name'] = boat
            new_row['img_path'] = img_path
            boat_data = pd.concat([boat_data, pd.DataFrame([new_row])], ignore_index=True)



    print("Done\n")
    if counter%100 == 1:
        print("saving")
        boat_data.to_csv(data_dir+"boat_data.csv" , index=False)
boat_data.to_csv(data_dir+"boat_data.csv" , index = False)

In [ ]:
pd.set_option('display.max_columns', None)
data_raw = pd.read_csv(data_dir+"boat_data.csv")
data_raw

## boat24.com

regex

In [ ]:
n_pages_tag = re.compile(r'\?page=(\d{3,})') # For getting the amount of pages on the domain

# name_and_url_tag = re.compile(r'__title\"><a href=\"(https://www\.boat24\.com.+)\"\stitle=\"(.+?)\"\s>')

# image_tag = re.compile(r'srcset=\"(https://[^\s]+?\.jpg) \d[^2]')

# name_tag = re.compile(r'\"blurb__title\"><a.+?title=\"(.+?)\"\s*')
# image_tag=re.compile(r'<div class=\"blurb__image-area\".+?srcset=\"(https://static\.b24\.co/fotos.+?\.jpg)|(https://www\.boat24\.com/img/icons/boat-types/mynd/type-1-230\.png)')
# url_tag = re.compile(r'<a href=\"(https://www\.boat24\.com/en/sailingboats/.+)')
url_name_image_tag = re.compile(r'data-link=\"(https://www\.boat24\.com/en/sailingboats/.{1,100}?\d\/)\"[^¤]*?title=\"(.{1,40})\"><div class=\"blurb__image-area\".+?srcset=\"(https://static\.b24\.co/fotos.+?\.jpg)')
image_tag = re.compile(r'src=\"(https://static\.b24\.co/fotos.+?\.jpg)')

In [ ]:
def scrape(url , tag):
    r = requests.get(url)
    retval = re.findall(tag , r.text)
    return list(zip(*retval))

In [ ]:
# sailboatdata_data = pd.read_csv(os.path.join(data_dir, "boat_data_clean.csv"))
# try:
#     boat24_data_raw = pd.read_csv(os.path.join(data_dir, "boat24_data_raw.csv"))
# except:
#     boat24_data_raw = pd.DataFrame(columns = ["url","name","img_url , html"])
# class Labeller:
#     def __init__(self , boat_names = Name_Classes.values(), embedding_size = 2**8):
#         self.boat_names = boat_names
#         self.onehot_map = {}
#         self.embedding_size = embedding_size
#         self.str2label = {v: k for k, v in Name_Classes.items()}
#         for boat in boat_names:
#             self.onehot_encode(boat)
#         self.label2embedding = {k: self.onehot_encode(v) for k, v in sorted(Name_Classes.items())}

#     def onehot_encode(self, boat_name):
#         retval = np.zeros(self.embedding_size)
#         for char in boat_name.upper():
#             try:
#                 retval += self.onehot_map[char]
#             except:
#                 embedding = np.zeros(2**8)
#                 embedding[len(self.onehot_map.keys())] = 1
#                 self.onehot_map[char] = embedding
#                 retval += self.onehot_map[char]
#         return retval
    
#     def add_label(self , boat_name , distance_threshold = 2):
#         try:
#             return True , self.str2label[boat_name.upper()] , 0
#         except:
#             embedding = self.onehot_encode(boat_name)
#             distances = np.linalg.norm(embedding - np.array(list(self.label2embedding.values())), axis=1)
#             return_label = np.argmin(distances)
#             if distances[return_label] < distance_threshold:
#                 # print(boat_name+" not found adding label" , Name_Classes[int(return_label)]+" with the distance" , distances[return_label])
#                 return True , return_label , distances[return_label]
#             else:
#                 # print(boat_name+" not found closest was" , Name_Classes[int(return_label)]+" with the distance" , distances[return_label])
#                 return False , return_label , distances[return_label]

In [ ]:
urls_test , names_test , images_test = scrape("https://www.boat24.com/en/sailingboats/?page=1" , url_name_image_tag)
print(urls_test)
print(names_test)
print(images_test)

In [ ]:
boat24_data_raw = pd.read_csv('data/boat24_data_raw.csv')
sailboatdata_data = pd.read_csv('data/boat_data_clean.csv')
boat_names = [Name_Classes[x] for x in sailboatdata_data['name'].unique()]

try:
    already_scraped = pd.read_csv(os.path.join(data_dir, "already_scraped.csv") , header=None)
except:
    already_scraped = pd.DataFrame(['dummy_url'])
try:
    boat24_data_clean = pd.read_csv(os.path.join(data_dir, "boat24_data_clean.csv"))
except:
    boat24_data_clean = pd.DataFrame(columns = sailboatdata_data.columns)

In [ ]:
init_request = requests.get("https://www.boat24.com/en/sailingboats/?page=0")
n_pages = re.findall(n_pages_tag , init_request.text)
print(int(n_pages[0]))

urls = []
names = []
images = []
html = []


for i in tqdm(range(0,int(n_pages[0])+1,20)):
    url_list , name_list , image_list = scrape("https://www.boat24.com/en/sailingboats/?page={}".format(i) , url_name_image_tag)
    urls += url_list
    names += name_list
    images += image_list
    html += [requests.get(url).text for url in url_list]

    time.sleep(0.1) #To not throw too many requests at the website
boat24_data_raw = pd.concat([boat24_data_raw , pd.DataFrame({"url":urls , "name":names , "img_url":images , "html":html})])
boat24_data_raw.drop_duplicates(subset=['url'] , keep='first')
boat24_data_raw['name'] = boat24_data_raw['name'].str.upper()
boat24_data_raw.to_csv(data_dir+"boat24_data_raw.csv" , index = False)
    

In [ ]:
# Could use UrlListCrawler with:
# from icrawler.builtin import UrlListCrawler to download using more threads
# see https://icrawler.readthedocs.io/en/latest/builtin.html

In [ ]:
boat24_data_clean

In [ ]:
boat24_data_raw = pd.read_csv(os.path.join(data_dir, "boat24_data_raw.csv"))
sailboatdata_data = pd.read_csv(os.path.join(data_dir, "boat_data_clean.csv"))

boat_names = [Name_Classes[x] for x in sailboatdata_data['name'].unique()]

labeller = Labeller(boat_names)
counter = 10
for path,subdirs,files in os.walk(os.path.join(os.getcwd(),img_dir,"boat24")):
    for file in files:
        counter += 1
save = False
print("Starting counter at" , counter)

for i in tqdm(range(len(boat24_data_raw))):
    found , name , distance = labeller.add_label([BeautifulSoup(boat24_data_raw['name'][i].lower() , 'html.parser').get_text()])
    if found and not (boat24_data_raw['url'][i] == already_scraped[0]).any():
        images = re.findall(image_tag , boat24_data_raw['html'][i])
        print('Setting' , BeautifulSoup(boat24_data_raw['name'][i] , 'html.parser').get_text() , 'to' , Name_Classes[name] , 'with distance' , distance)
        for image in images:
            file_name = "boat24_"+str(counter).zfill(6)+".jpg"
            img_subdir = os.path.join(img_dir,"boat24", Name_Classes[name])
            if not os.path.exists(img_subdir):
                os.makedirs(img_subdir)
            try:
                img_request.urlretrieve(image,os.path.join(img_subdir,file_name))
                row = sailboatdata_data[sailboatdata_data['name'] == name].iloc[0].to_frame().T
                row['img_path'] = file_name
                row['img_url'] = image
                row['image_hash'] = None
                row['original_label'] = BeautifulSoup(boat24_data_raw['name'][i] , 'html.parser').get_text()
                boat24_data_clean = pd.concat([boat24_data_clean , row])
                counter += 1
            except:
                print("Error downloading image",image)
            if counter % 10 == 0:
                save = True
    already_scraped = pd.concat([already_scraped , pd.DataFrame([boat24_data_raw['url'][i]])])
    if save:
        boat24_data_clean.to_csv(os.path.join(data_dir,"boat24_data_clean.csv") , index=False)
        already_scraped.to_csv(os.path.join(data_dir,"already_scraped.csv") , index=False , header=False)
        save = False
boat24_data_clean.to_csv(os.path.join(data_dir,"boat24_data_clean.csv") , index=False)
already_scraped.to_csv(os.path.join(data_dir,"already_scraped.csv") , index=False , header=False)

print("Checking if boat24_data_clean only contains uniques : " , boat24_data_clean.value_counts('img_path').sum() == len(boat24_data_clean))



## Image Search

In [ ]:
boat_data_clean = pd.read_csv(data_dir+"sailboatdata_data_hashed_cleaned.csv")
from icrawler.builtin import BingImageCrawler

boat_names = [Name_Classes[x] for x in boat_data_clean['name'].unique()]
already_found = os.walk(os.path.join(os.getcwd(),img_dir,"image_search_50")).__next__()[1]
for boat in boat_names:
    if boat not in already_found:
        path = os.path.join(img_dir,"image_search_50",boat)
        if not(os.path.exists(path)):
            print("Making dir" , path)
            makedirs(path)
        bing_Crawler = BingImageCrawler(storage = {'root_dir': path} , downloader_threads = 16)
        bing_Crawler.crawl(keyword = boat + " sailboat", max_num = 50 , filters = {"type":"photo"})



In [ ]:
# counter = 0
sailboatdata_data = pd.read_csv(data_dir+"boat_data_clean.csv")
label2id = {Name_Classes[x]:x for x in Name_Classes}
image_search_data = pd.DataFrame(columns = sailboatdata_data.columns)
for dir in tqdm(os.walk(os.path.join(os.getcwd(),img_dir,"image_search")).__next__()[1]):
    rows = []
    for file in os.listdir(os.path.join(os.getcwd(),img_dir,"image_search",dir)):
        if file.endswith(".jpg"):
            # file_name = "image_search" + str(counter).zfill(6) + ".jpg"
            # counter += 1
            # os.rename(os.path.join(os.getcwd(),img_dir,"image_search",dir,file) , os.path.join(os.getcwd(),img_dir,"image_search",dir,file_name))
            try:
                row = sailboatdata_data[sailboatdata_data['name'] == label2id[dir]].iloc[0].to_frame().T
            except:
                row = sailboatdata_data[sailboatdata_data['name'] == label2id[dir+"."]].iloc[0].to_frame().T
            row['img_path'] = dir+"_"+file
            rows += [row]
    try:
        image_search_data = pd.concat([image_search_data , pd.concat(rows)])
    except:
        print("Error with dir",dir)
image_search_data.to_csv(os.path.join(data_dir,"image_search_data_50.csv") , index=False)

# Cleaning and Data Processing

## Initial HTML cleanup

In [ ]:
units_cleaner_tag = re.compile(r'([\d.]+)(\s*m|\s*kg)(<.*>)*')

def clean_row(row):
  cleaned_row = []
  for i in range(len(row)):
    if i != 32:
      try:
        cleaned_row.append(float(re.sub(units_cleaner_tag , r'\g<1>' , row[i].replace(',','') , )))
      except:
        cleaned_row.append(row[i])
    else:
      cleaned_row.append(row[i])
  return cleaned_row

In [ ]:
cleaned_data = []
for i , row in data_raw.iterrows():
  cleaned_data.append(clean_row(row))

In [ ]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)
data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)

In [ ]:
data_clean = pd.read_csv(data_dir+"boat_data_clean.csv")

In [ ]:
row_clean_tag = re.compile(r'[^\w\s/&]')
for column in data_clean.columns:
    if (data_clean[column].dtype == "object" or data_clean[column].dtype == "category") and not column == "Download Boat Record" and not column == "url" and not column == "img_path":
        data_clean[column] = data_clean[column].str.strip().str.rstrip('.').str.lower()
        data_clean[column] = data_clean[column].str.replace(row_clean_tag , '')
        # data_clean[column] = data_clean[column].astype('category')
        # pd.DataFrame(data_clean[column].cat.categories).to_csv(data_dir+"labels/"+column.replace(" ","_")+".txt" , index = False , header = False)
        # data_clean[column] = data_clean[column].cat.codes

## Image Hashing

In [ ]:
# Average hash computation.
# See:
# https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html
def average_hasher(image):
  # Read image.
  img = PIL.Image.open(image)

  # Simplify image by reducing its size and colors.
  pixels = img.convert("L").resize((8, 8), Image.ANTIALIAS).getdata()

  # Get the average pixel value.
  sum = 0.0
  for p in pixels: sum += p
  mean = sum / 64

  # Generate the hash by comparing each pixel's value to the average.
  bits = 0
  for p in pixels:
    bits <<= 1
    if p > mean: bits |= 1

  return "%016x" % bits, img.width, img.height

In [ ]:
def hash_images(dir , data , existing_hashes = []):
    image_hashes = []
    found_duplicates = 0
    for i , row in tqdm(data.iterrows()):
        try:
            img_hash = average_hasher(os.path.join(img_dir , dir , data['img_path'][i] ))[0]
        except:
            try:
                img_hash = average_hasher(os.path.join(img_dir , dir , Name_Classes[data['name'][i]] ,data['img_path'][i] ))[0]
            except:
                print("Error with ",os.path.join(img_dir , dir , Name_Classes[data['name'][i]] ,data['img_path'][i]))
                img_hash = None
        if img_hash not in existing_hashes:
            image_hashes.append(img_hash)
        else:
            if found_duplicates <= 10:
                print("Found duplicate",img_hash)
            elif found_duplicates < 10:
                print("Found over 10 duplicates. Will not print more...")
            found_duplicates += 1
            image_hashes.append(None)
    print("Found",found_duplicates,"duplicates")
    return image_hashes

In [ ]:
sailboatdata_data = pd.read_csv(data_dir+"boat_data_clean.csv")
sailboatdata_data['image_hash'] = hash_images("sailboatdata" , sailboatdata_data)
# Remove duplicate hashes.
sailboatdata_data = sailboatdata_data.dropna(subset=['image_hash'])
print("Number of unique hashes:",len(sailboatdata_data))
sailboatdata_data.to_csv(data_dir+"sailboatdata_data_hashed.csv" , index = False)


In [ ]:
boat24_data = pd.read_csv(data_dir+"boat24_data_clean.csv")
boat24_data['image_hash'] = hash_images("boat24" , boat24_data)
# Remove duplicate hashes.
boat24_data = boat24_data.dropna(subset=['image_hash'])
print("Number of unique hashes:",len(boat24_data))
boat24_data.to_csv(data_dir+"boat24_data_hashed.csv" , index = False)

In [ ]:
image_search_data = pd.read_csv(data_dir+"image_search_data.csv")
image_search_data['image_hash'] = hash_images("image_search" , image_search_data , sailboatdata_data['image_hash'].tolist())
# Remove duplicate hashes.
image_search_data = image_search_data.dropna(subset=['image_hash'])
print("Number of unique hashes:",len(image_search_data))
image_search_data.to_csv(data_dir+"image_search_data_hashed.csv" , index = False)

## Label Grouping

### Construction

In [ ]:
def clean_construction(data):
    labels = []
    for i in range(len(data)):
        label = Construction_Classes[data['Construction'][i]].lower().strip().replace("."," ").replace("/"," ").replace(","," ")
        if label not in labels:
            labels.append(label)
    label_map = {
                    "Fibreglass" : [],
                    "GRP" : [],
                    "Carbon" : [],
                    "Wood" : [],
                    "Metal" : [],
                    "Plastic" : [],
                    "NaN" : [],
                
                }
    banned_words = ["or" , "nan" , "none" , "unknown" , "other" , "various" , "various" , "unknown" , "n/a" , "not applicable"]
    fibreglass_contains = ["fibreglass" , "fiberglass" , "glass" , "glassfiber" , "glass fibre" , "glassfiber" , "fg"]
    grp_contains = ["grp"]
    carbon_contains = ["carbon" , "carbonfiber" , "carbon fibre" , "carbonfiber"]
    wood_contains = ["wood" , "mahogany" , "teak" , "oak" , "cedar" , "pine" , "fir" , "spruce" , "balsa" , "plywood" , "ply wood"]
    metal_contains = ["metal" , "steel" , "aluminium" , "aluminum" , "iron" , "bronze" , "copper" , "lead" , "brass" , "stainless steel" , "stainlesssteel" , "stainlesssteel" , "alu"]
    plastic_contains = ["plastic" , "pvc" , "polyester" , "polyethylene" , "polypropylene" , "polyurethane" , "polyvinyl chloride" , "polyvinylchloride" , "polyvinylchloride"]
    for i in range(len(labels)):
        if labels[i] not in label_map:
            # First check if label contains any banned words.
            if any(word in labels[i] for word in banned_words):
                label_map["NaN"].append(labels[i])
            # Then check if label contains any carbon words.
            elif any(word in labels[i] for word in carbon_contains):
                label_map["Carbon"].append(labels[i])
            # Then check if label contains any grp words.
            elif any(word in labels[i] for word in grp_contains):
                label_map["GRP"].append(labels[i])
            # Then check if label contains any fibreglass words.
            elif any(word in labels[i] for word in fibreglass_contains):
                label_map["Fibreglass"].append(labels[i])
            # Then check if label contains any wood words.
            elif any(word in labels[i] for word in wood_contains):
                label_map["Wood"].append(labels[i])
            # Then check if label contains any metal words.
            elif any(word in labels[i] for word in metal_contains):
                label_map["Metal"].append(labels[i])
            # Then check if label contains any plastic words.
            elif any(word in labels[i] for word in plastic_contains):
                label_map["Plastic"].append(labels[i])
            else:
                label_map["NaN"].append(labels[i])

    # Reverse the label map.
    reverse_label_map = {}
    for key in label_map:
        for value in label_map[key]:
            reverse_label_map[value] = key
    reverse_label_map = {k: v for k, v in sorted(reverse_label_map.items(), key=lambda item: item[1])}


    for i in range(len(data)):
        data['Construction'][i] = reverse_label_map[Construction_Classes[data['Construction'][i]].lower().strip().replace("."," ").replace("/"," ").replace(","," ")]

In [ ]:
def clean_ballast_type(data):
    labels = []
    for i in range(len(data)):
        label = Ballast_Type_Classes[data['Ballast Type'][i]].lower().replace("."," ").replace("*"," ").strip()
        if label not in labels:
            labels.append(label)
    label_map = {
                    "Lead" : [],
                    "Iron" : [],
                    "Concrete" : [],
                    "Steel" : [],
                    "Water" : [],
                    "Stone" : [],
                    "Mix" : [],
                    "NaN" : [],
                }
    mix_contains = ["mix" , "mixed" , "or" , "/" , "and" , "with" , "varies" , "anything" , "%" , "+" , "," , ";" , ":"]
    lead_contains = ["lead" , "lea" , ]
    iron_contains = ["iron"]
    concrete_contains = ["concrete" , "cement"]
    steel_contains = ["steel" , "SS"]
    water_contains = ["water"]
    stone_contains = ["stone"]
    for i in range(len(labels)):
        if labels[i] not in label_map:
            # First check if label contains any mix words.
            if any(word in labels[i] for word in mix_contains):
                label_map["Mix"].append(labels[i])
            # Then check if label contains any lead words.
            elif any(word in labels[i] for word in lead_contains):
                label_map["Lead"].append(labels[i])
            # Then check if label contains any iron words.
            elif any(word in labels[i] for word in iron_contains):
                label_map["Iron"].append(labels[i])
            # Then check if label contains any concrete words.
            elif any(word in labels[i] for word in concrete_contains):
                label_map["Concrete"].append(labels[i])
            # Then check if label contains any steel words.
            elif any(word in labels[i] for word in steel_contains):
                label_map["Steel"].append(labels[i])
            # Then check if label contains any water words.
            elif any(word in labels[i] for word in water_contains):
                label_map["Water"].append(labels[i])
            # Then check if label contains any stone words.
            elif any(word in labels[i] for word in stone_contains):
                label_map["Stone"].append(labels[i])
            else:
                label_map["NaN"].append(labels[i])

    # Reverse the label map.
    reverse_label_map = {}
    for key in label_map:
        for value in label_map[key]:
            reverse_label_map[value] = key
    reverse_label_map = {k: v for k, v in sorted(reverse_label_map.items(), key=lambda item: item[1])}

    for i in range(len(data)):
        data['Ballast Type'][i] = reverse_label_map[Ballast_Type_Classes[data['Ballast Type'][i]].lower().replace("."," ").replace("*"," ").strip()]

In [ ]:
sailboatdata_data = pd.read_csv(data_dir+"sailboatdata_data_hashed.csv")
boat24_data = pd.read_csv(data_dir+"boat24_data_hashed.csv")
image_search_data = pd.read_csv(data_dir+"image_search_data_hashed.csv")
data_list = [sailboatdata_data , boat24_data , image_search_data]

for data in data_list:
    clean_ballast_type(data)
    clean_construction(data)

# Make a label2id map for ballast type. and construction.
ballast_types = (sailboatdata_data['Ballast Type'].dropna().unique().tolist())
construction_types = (sailboatdata_data['Construction'].dropna().unique().tolist())
ballast_types.remove("NaN")
construction_types.remove("NaN")
ballast_types = ["NaN"] + ballast_types
construction_types = ["NaN"] + construction_types

ballast_type_id2label = {v : k for k , v in zip(ballast_types , range(-1 , len(ballast_types)))}
construction_id2label = {v : k for k , v in zip(construction_types , range(-1 , len(construction_types)))}

# Make an id2label map for ballast type. and construction.
ballast_type_label2id = {v : k for k , v in ballast_type_id2label.items()}
construction_label2id = {v : k for k , v in construction_id2label.items()}

# Save the label2id and id2label maps.
with open(data_dir+"ballast_type_label2id.json" , "w") as f:
    json.dump(ballast_type_label2id , f)
with open(data_dir+"ballast_type_id2label.json" , "w") as f:
    json.dump(ballast_type_id2label , f)
with open(data_dir+"construction_label2id.json" , "w") as f:
    json.dump(construction_label2id , f)
with open(data_dir+"construction_id2label.json" , "w") as f:
    json.dump(construction_id2label , f)

# Map the labels to ids.
for data in data_list:
    for i in range(len(data)):
        if data['Ballast Type'][i] is not None:
            data['Ballast Type'][i] = ballast_type_label2id[data['Ballast Type'][i]]
        if data['Construction'][i] is not None:
            data['Construction'][i] = construction_label2id[data['Construction'][i]]

# Save the data.
sailboatdata_data.to_csv(data_dir+"sailboatdata_data_hashed_cleaned.csv" , index=False)
boat24_data.to_csv(data_dir+"boat24_data_hashed_cleaned.csv" , index=False)
image_search_data.to_csv(data_dir+"image_search_data_hashed_cleaned.csv" , index=False)

# Splitting

In [ ]:
data_train , data_test = train_test_split(data_clean , test_size = 0.2 , random_state = 42)


In [ ]:
train_dir = "data/train/"
test_dir = "data/test/"

if not(os.path.exists(train_dir)):
    print("Making dir" , train_dir)
    makedirs(train_dir)

if not(os.path.exists(test_dir)):
    print("Making dir" , test_dir)
    makedirs(test_dir)


for image in data_train['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(train_dir+image)
    except:
        print("Image not found" , image)
        data_train = data_train[data_train['img_path'] != image]

for image in data_test['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(test_dir+image)
    except:
        print("Image not found" , image)
        data_test = data_test[data_test['img_path'] != image]


In [ ]:
data_train.to_csv(data_dir+"boat_data_train.csv" , index = False)
data_test.to_csv(data_dir+"boat_data_test.csv" , index = False)

# Label embedding

In [ ]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)

col_list = ['Hull Type' , 'Rigging Type' , 'Construction' , 'Ballast Type' , 'Designer' , 'name']

for column in data_clean.columns:
    if column in col_list:
        path = data_dir+"/labels/"+column.replace(" ","_")
        data_clean[column] = data_clean[column].astype('category')
        classes = dict( zip(data_clean[column] , data_clean[column].cat.codes))
        with open(path+".json", "w") as outfile:
            json.dump(classes, outfile , indent = 4)
        data_clean[column] = data_clean[column].cat.codes

data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)
data_train , data_test = train_test_split(data_clean , test_size = 0.2 , random_state = 42)
data_train.to_csv(data_dir+"boat_data_train.csv" , index = False)
data_test.to_csv(data_dir+"boat_data_test.csv" , index = False)

In [ ]:
# file_path = "images/00002.jpg"
# df = data_train
# idx = df.index[(df['img_path'] == file_path.split("/")[1])].values[0]

# print(
# Hull_Type_Classes[df['Hull Type'][idx]],[df['Hull Type'][idx]],"\n"+
# Rigging_Type_Classes[df['Rigging Type'][idx]],[df['Rigging Type'][idx]],"\n"+
# Construction_Classes[df['Construction'][idx]],[df['Construction'][idx]],"\n"+
# Ballast_Type_Classes[df['Ballast Type'][idx]],[df['Ballast Type'][idx]],"\n"+
# Designer_Classes[df['Designer'][idx]],[df['Designer'][idx]],"\n"+
# Name_Classes[df['name'][idx]],[df['name'][idx]],"\n"
# )

In [ ]:
labels = ['Hull Type' , 'Rigging Type' , 'Construction' , 'Ballast Type' , 'Designer' , 'name']
for label in labels:
    sailboatdata_data = pd.read_csv("data/boat_data_clean.csv")
    sailboatdata_data = sailboatdata_data.dropna(subset=[label])
    # drop labels with less than 2 entries
    
    sailboatdata_data = sailboatdata_data.groupby(label).filter(lambda x: len(x) > 1)
    sailboatdata_data = sailboatdata_data.drop_duplicates(subset=[label])
    print(label,sailboatdata_data.__len__())